#### Utils

In [1]:
from scipy.stats import poisson,norm
from math import log
from scipy.integrate import simpson
from scipy.optimize import minimize
from functools import partial
from scipy.integrate import quad
import numpy as np
from matplotlib import pyplot as plt
from scipy.integrate import cumulative_trapezoid

def model_poisson(n,s,b):
    return poisson(s+b).pmf(n)

def prior_b(b,bmu,sigmab):
    return norm(loc=bmu,scale=sigmab).pdf(b)

def constraint_b(b,bmu,sigmab):
    return norm(loc=bmu,scale=sigmab).pdf(b)

# prior_b and constraint_b practically equal, conceptually different
def model_constraint_bayes(n,s,b,bbar,sigmab):
    return model_poisson(n,s,b)*prior_b(b,bbar,sigmab)
    
def model_constraint_freq(n,s,b,bbar,sigmab):
    return model_poisson(n,s,b)*constraint_b(bbar,b,sigmab)

def model_poisson_2d(n,x):
    s = x[0]
    b = x[1]
    return model_poisson(n,s,b)
    
def model_constraint_2d_freq(n,x,bbar,sigmab):
    s = x[0]
    b = x[1]
    return model_constraint_freq(n,s,b,bbar,sigmab)

def nll(model,n,s,b,**kwargs):
    return -np.log(model(n,s,b,**kwargs))

def nll_2d(model,n,x,**kwargs):
    return -np.log(model(n=n,x=x,**kwargs))

def integral_2d(f,x,y):
    zz = f(x.reshape(-1,1),y.reshape(1,-1))
    return simpson([simpson(zz_y,y) for zz_y in zz],x) 
    
def posterior(model,n,b=None,s=np.linspace(0,10,101),profile_b=False,TwoD = False,**kwargs):
    if type(b) is int or type(b) is float: # fixed b
        integralmodel = quad(partial(model,n,b=b,**kwargs),0,np.inf)[0]
        return model(n,s,b,**kwargs)/integralmodel
    else: # b not fixed
        if profile_b: # optimise b
            post = []
            for s_i in s:
                res=minimize(partial(nll,model,n,s_i,**kwargs),x0=(3,),bounds=[(0,np.inf)])
                optimalb = res.x[0]
                post.append(model(n=n,s=s_i,b=optimalb,**kwargs))
            post = np.array(post)
            integralmodel = simpson(post,s)
            return post/integralmodel
        else: # integrate b
            post = []
            integral = integral_2d(partial(model,n,**kwargs),s,b)
            if TwoD: # Posterior 2D (s,b)
                return model(n,s = s.reshape(1,-1),b = b.reshape(-1,1),**kwargs)/integral
            else: # Posterior 1D (s) / integrate b
                integral_b = simpson(model(n,s.reshape(-1,1),b.reshape(1,-1),**kwargs),b,axis=1)
                return integral_b/integral

def testStatistic(model,n,s,b,bestNLL,**kwargs):
    return 2*(nll(model=model,n=n,s=s,b=b,**kwargs)-bestNLL)

In [2]:
from matplotlib import pyplot as plt
plt.rcParams['figure.figsize'] = (12,8)
plt.rc('xtick',labelsize=15)
plt.rc('ytick',labelsize=15)
plt.rc('legend',fontsize=15)
plt.rc('font',size=15)


#### Python

In [3]:
from scipy.stats import chi2
from random import gauss
expected_n = 200
expected_bbar = 150
sigmab = 30
Ntoys = 10000 #Because 10000 takes 2 hours
upper_limits = []
limit = chi2(1).ppf(0.9) # 95% one sided CL
s = np.linspace(0.5,150,20)
# plt.figure(figsize=(50,25))

for i in range(Ntoys):
    n = np.random.poisson(expected_n, 1)
    bbar = gauss(expected_bbar, sigmab)    
    res = minimize(partial(nll_2d,model_constraint_2d_freq,n,bbar=bbar,sigmab=sigmab),x0=(2,3),bounds=[(0,np.inf),(0,np.inf),])
    bestNLL = res.fun
    t = np.array([])
    for si in s:
        res = minimize(partial(nll,model_constraint_freq,n,si,bbar=bbar,sigmab=sigmab),x0=(3,),bounds=[(0,np.inf),])
        t = np.append(t,testStatistic(model=model_constraint_freq,n=n,s=si,b=res.x[0],bestNLL=bestNLL,bbar=bbar,sigmab=sigmab))
    # plt.subplot(13,10,i+1)
    # plt.plot(s,t,label=r'$\sigma_b=%.2f$'%sigmab,lw=1)
    idx = (t>limit).argmax()
    upper_limit = 0.5*(s[idx]+s[idx-1])
    # print('pper limit = %.2f'%(upper_limit))
    upper_limits.append(upper_limit)
    if (i%100 == 0):
        print(i)
    # plt.vlines(upper_limit,0,limit,lw=1)
# plt.xlabel('s',fontsize=15)
# plt.axhline(limit,color='black',lw=1)
# plt.legend(loc='best')

0


/tmp/ipykernel_26416/1736631009.py:38: RuntimeWarning: divide by zero encountered in log
  return -np.log(model(n,s,b,**kwargs))
/home/alberto/.local/lib/python3.10/site-packages/scipy/optimize/_numdiff.py:576: RuntimeWarning: invalid value encountered in subtract
  df = fun(x) - f0


100
200
300
400
500
600
700
800
900
1000
1100
1200


/tmp/ipykernel_26416/1736631009.py:41: RuntimeWarning: divide by zero encountered in log
  return -np.log(model(n=n,x=x,**kwargs))


1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
7100
7200
7300
7400
7500
7600
7700
7800
7900
8000
8100
8200
8300
8400
8500
8600
8700
8800
8900
9000
9100
9200
9300
9400
9500
9600
9700
9800
9900


In [4]:
upper_limits = np.array(upper_limits)
coverage = np.size(upper_limits[upper_limits>50])/np.size(upper_limits)
print(str(coverage*100))

95.39


In [7]:
from scipy.stats import chi2
expected_n = 20
expected_bbar = 15
limit = chi2(1).ppf(0.9) # 95% one sided CL
s = np.linspace(0,40,30)
sigmab = 9.5
Ntoys = 10000 #Because 10000 takes 2 hours
upper_limits = []

for i in range(Ntoys):
    n = np.random.poisson(expected_n, 1)
    bbar = gauss(expected_bbar, sigmab)    
    res = minimize(partial(nll_2d,model_constraint_2d_freq,n,bbar=bbar,sigmab=sigmab),x0=(2,3),bounds=[(0,np.inf),(0,np.inf),])
    bestNLL = res.fun
    t = np.array([])
    for si in s:
        res = minimize(partial(nll,model_constraint_freq,n,si,bbar=bbar,sigmab=sigmab),x0=(3,),bounds=[(0,np.inf),])
        t = np.append(t,testStatistic(model=model_constraint_freq,n=n,s=si,b=res.x[0],bestNLL=bestNLL,bbar=bbar,sigmab=sigmab))
    # plt.subplot(13,10,i+1)
    # plt.plot(s,t,label=r'$\sigma_b=%.2f$'%sigmab,lw=1)
    idx = (t>limit).argmax()
    upper_limit = 0.5*(s[idx]+s[idx-1])
    # print('pper limit = %.2f'%(upper_limit))
    upper_limits.append(upper_limit)
    if (i%100 == 0):
        print(i)

0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
7100
7200
7300
7400
7500
7600
7700
7800
7900
8000
8100
8200
8300
8400
8500
8600
8700
8800
8900
9000
9100
9200
9300
9400
9500
9600
9700
9800
9900


In [6]:
upper_limits = np.array(upper_limits)
coverage = np.size(upper_limits[upper_limits>5])/np.size(upper_limits)
print(str(coverage*100))

99.65818458032662
